In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import requests
from bs4 import BeautifulSoup

ISBN = 9789869109321

url = f"http://webpac.lib.ntpu.edu.tw/search.cfm?m=ss&k0={ISBN}&t0=k&c0=and"
my_options = Options()
my_options.add_argument("--headless")
driver = webdriver.Chrome(options=my_options)
driver.get(url)

html_text = driver.page_source
soup = BeautifulSoup(html_text, "html.parser")

hrefs = []
list_boxes = soup.find_all("div", "list_box")
for list_box in list_boxes:
    hrefs.append("http://webpac.lib.ntpu.edu.tw/" + list_box.find("a", "btn")["href"])
driver.quit()

df_final = pd.DataFrame()
for url in hrefs:
    # 進入頁面爬取表格
    dfs = pd.read_html(url, encoding="utf-8")
    df_temp = dfs[0]
    
    # 新增欄位《館藏地》
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text, "html.parser")
    place_dad = soup.find("div", "book_area").find("h2").text
    place_son = soup.find("div", "book_area").find("h3").text
    df_temp.insert(1, "館藏地", [place_dad + place_son for i in range(df_temp.shape[0])])
    
    # 合併表格
    df_final = pd.concat([df_final, df_temp], axis=0, ignore_index=True)
df_final

# 新增欄位《圖書館》
df_final.insert(0, "圖書館", ["國立臺北大學" for i in range(df_final.shape[0])])
# 刪除不要的欄位
for removed in ["登錄號", "資料類型", "使用類型", "附件", "預約人數", "Unnamed: 7"]:
    df_final.pop(removed)
df_final